In [29]:
%pip install opencv-python
%pip install cvlib
%pip install scikit-learn
%pip install joblib
%pip install pandas
%pip install numpy
# If not installed already (and for Python 3.11):
%pip install tensorflow==2.20.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [23]:
import cv2
import cvlib as cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
import pandas as pd
import numpy as np
import os

In [24]:
path = r"/Users/gabrielmercado/Documents/MLSN-Team-6/images"

def load_images_from_folder(base_path):
    data = {'image' : [], 'emotion': []}

    for emotion in os.listdir(base_path):
        emotion_folder = os.path.join(base_path, emotion)

        if not os.path.isdir(emotion_folder):
            continue

        for file in os.listdir(emotion_folder):
            if file.lower().endswith(('.png','.jpg','.jpeg')):
                img_path = os.path.join(emotion_folder,file)
                img = cv2.imread(img_path)

                if img is None:
                    continue

                img = cv2.resize(img, (100, 100))
                data['image'].append(img)
                data['emotion'].append(emotion)

    return pd.DataFrame(data)



In [25]:
train_data = load_images_from_folder(path)
train_data

X_train, X_test, Y_train, Y_test = train_test_split(train_data['image'],train_data['emotion'],test_size= .2, random_state=42)

X_train_flat = [img.flatten() for img in X_train]
X_test_flat = [img.flatten() for img in X_test]



In [26]:
model = Pipeline([
    ('classifier',RandomForestClassifier(n_estimators= 100))

])

model.fit(X_train_flat,Y_train)




Pipeline(steps=[('classifier', RandomForestClassifier())])

In [27]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test_flat)
acc = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {acc:.4f}")


from sklearn.metrics import classification_report

print(classification_report(Y_test,Y_pred))

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test,Y_pred)
print(cm)

Accuracy: 0.5915
              precision    recall  f1-score   support

     Neutral       0.54      0.70      0.61       171
       happy       0.60      0.68      0.64       180
         sad       0.38      0.08      0.13        75
    surprise       0.70      0.62      0.66       137

    accuracy                           0.59       563
   macro avg       0.55      0.52      0.51       563
weighted avg       0.58      0.59      0.57       563

[[120  34   6  11]
 [ 40 122   2  16]
 [ 35  24   6  10]
 [ 26  24   2  85]]


In [28]:
model_filename = 'emotion_detection_model.pkl'
joblib.dump(model, model_filename)





['emotion_detection_model.pkl']